In [1]:
# run this to shorten the data import from the files
path_data = '/home/nero/Documents/Estudos/DataCamp/Python/Introduction_to_PySpark/datasets/'


In [2]:
# start spark session

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

23/07/10 09:00:56 WARN Utils: Your hostname, nero resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface wlp2s0)
23/07/10 09:00:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/10 09:00:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
airports = spark.read.csv(path_data + "/airports.csv", header=True)
planes = spark.read.csv(path_data + 'planes.csv', header=True)
flights = spark.read.csv(path_data + 'flights_small.csv', header=True)

In [4]:
airports.createOrReplaceTempView('airports')
planes.createOrReplaceTempView('planes')
flights.createOrReplaceTempView('flights')

In [5]:
# exercise 01

"""
Join the DataFrames

In the next two chapters you'll be working to build a model that predicts whether or not a flight will be delayed based on the flights data we've been working with. This model will also include information about the plane that flew the route, so the first step is to join the two tables: flights and planes!
"""

# Instructions

"""

    First, rename the year column of planes to plane_year to avoid duplicate column names.
    Create a new DataFrame called model_data by joining the flights table with planes using the tailnum column as the key.

"""

# solution

# Rename year column
planes = planes.withColumnRenamed('year','plane_year')

# Join the DataFrames
model_data = flights.join(planes, on='tailnum', how="leftouter")

#----------------------------------#

# Conclusion

"""
Awesome work! You're one step closer to a model!
"""

"\nAwesome work! You're one step closer to a model!\n"

In [6]:
model_data.show(5)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year|                type|manufacturer|   model|engines|seats|speed|   engine|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|      2011|Fixed wing multi ...|      AIRBUS|A320-214|      2|  182|   NA|Turbo-fan|
| N559AS|2014|    1| 22|    1040|        5|    1505|        5|     AS|   851|   SEA| HNL|     360|    2677|  10|    40|      2006|Fixed wing multi ...|      BOEING| 737-890|      2|  1

In [7]:
# exercise 02

"""
String to integer

Now you'll use the .cast() method you learned in the previous exercise to convert all the appropriate columns from your DataFrame model_data to integers!

To convert the type of a column using the .cast() method, you can write code like this:

dataframe = dataframe.withColumn("col", dataframe.col.cast("new_type"))
"""

# Instructions

"""
Use the method .withColumn() to .cast() the following columns to type "integer". Access the columns using the df.col notation:

    model_data.arr_delay
    model_data.air_time
    model_data.month
    model_data.plane_year
"""

# solution

# Cast the columns to integers
model_data = model_data.withColumn("arr_delay", model_data.arr_delay.cast('integer'))
model_data = model_data.withColumn("air_time", model_data.air_time.cast('integer'))
model_data = model_data.withColumn("month", model_data.month.cast('integer'))
model_data = model_data.withColumn("plane_year", model_data.plane_year.cast('integer'))

#----------------------------------#

# Conclusion

"""
Awesome! You're a pro at converting columns!
"""

"\nAwesome! You're a pro at converting columns!\n"

In [8]:
# exercise 03

"""
Create a new column

In the last exercise, you converted the column plane_year to an integer. This column holds the year each plane was manufactured. However, your model will use the planes' age, which is slightly different from the year it was made!
"""

# Instructions

"""

    Create the column plane_age using the .withColumn() method and subtracting the year of manufacture (column plane_year) from the year (column year) of the flight.

"""

# solution

# Create the column plane_age
model_data = model_data.withColumn("plane_age", model_data.year - model_data.plane_year)

#----------------------------------#

# Conclusion

"""
Great work! Now you have one more variable to include in your model.
"""

'\nGreat work! Now you have one more variable to include in your model.\n'

In [9]:
# exercise 04

"""
Making a Boolean

Consider that you're modeling a yes or no question: is the flight late? However, your data contains the arrival delay in minutes for each flight. Thus, you'll need to create a boolean column which indicates whether the flight was late or not!
"""

# Instructions

"""

    Use the .withColumn() method to create the column is_late. This column is equal to model_data.arr_delay > 0.
    Convert this column to an integer column so that you can use it in your model and name it label (this is the default name for the response variable in Spark's machine learning routines).
    Filter out missing values (this has been done for you).

"""

# solution

# Create is_late
model_data = model_data.withColumn("is_late", model_data.arr_delay > 0)

# Convert to an integer
model_data = model_data.withColumn("label", model_data.is_late.cast('integer'))

# Remove missing values
model_data = model_data.filter("arr_delay is not NULL and dep_delay is not NULL and air_time is not NULL and plane_year is not NULL")

#----------------------------------#

# Conclusion

"""
Awesome! Now you've defined the column that you're going to use as the outcome in your model.
"""

"\nAwesome! Now you've defined the column that you're going to use as the outcome in your model.\n"

In [11]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [12]:
# exercise 05

"""
Carrier

In this exercise you'll create a StringIndexer and a OneHotEncoder to code the carrier column. To do this, you'll call the class constructors with the arguments inputCol and outputCol.

The inputCol is the name of the column you want to index or encode, and the outputCol is the name of the new column that the Transformer should create.
"""

# Instructions

"""

    Create a StringIndexer called carr_indexer by calling StringIndexer() with inputCol="carrier" and outputCol="carrier_index".
    Create a OneHotEncoder called carr_encoder by calling OneHotEncoder() with inputCol="carrier_index" and outputCol="carrier_fact".

"""

# solution

# Create a StringIndexer
carr_indexer = StringIndexer(inputCol='carrier', outputCol='carrier_index')

# Create a OneHotEncoder
carr_encoder = OneHotEncoder(inputCol="carrier_index",outputCol="carrier_fact")

#----------------------------------#

# Conclusion

"""
Fantastic work! You're ready to include this column in your model now!
"""

"\nFantastic work! You're ready to include this column in your model now!\n"

In [15]:
# exercise 06

"""
Destination

Now you'll encode the dest column just like you did in the previous exercise.
"""

# Instructions

"""

    Create a StringIndexer called dest_indexer by calling StringIndexer() with inputCol="dest" and outputCol="dest_index".
    Create a OneHotEncoder called dest_encoder by calling OneHotEncoder() with inputCol="dest_index" and outputCol="dest_fact".

"""

# solution

# Create a StringIndexer
dest_indexer = StringIndexer(inputCol="dest", outputCol="dest_index")

# Create a OneHotEncoder
dest_encoder = OneHotEncoder(inputCol="dest_index", outputCol="dest_fact")

#----------------------------------#

# Conclusion

"""
Perfect! You're all done messing with factors.
"""

"\nPerfect! You're all done messing with factors.\n"

In [16]:
from pyspark.ml.feature import VectorAssembler

In [17]:
# exercise 07

"""
Assemble a vector

The last step in the Pipeline is to combine all of the columns containing our features into a single column. This has to be done before modeling can take place because every Spark modeling routine expects the data to be in this form. You can do this by storing each of the values from a column as an entry in a vector. Then, from the model's point of view, every observation is a vector that contains all of the information about it and a label that tells the modeler what value that observation corresponds to.

Because of this, the pyspark.ml.feature submodule contains a class called VectorAssembler. This Transformer takes all of the columns you specify and combines them into a new vector column.
"""

# Instructions

"""
Create a VectorAssembler by calling VectorAssembler() with the inputCols names as a list and the outputCol name "features".

    The list of columns should be ["month", "air_time", "carrier_fact", "dest_fact", "plane_age"].
"""

# solution

# Make a VectorAssembler
vec_assembler = VectorAssembler(inputCols=["month", "air_time", "carrier_fact", "dest_fact", "plane_age"], outputCol="features")

#----------------------------------#

# Conclusion

"""
Good job! Your data is all assembled now.
"""

'\nGood job! Your data is all assembled now.\n'

In [18]:
# exercise 08

"""
Create the pipeline

You're finally ready to create a Pipeline!

Pipeline is a class in the pyspark.ml module that combines all the Estimators and Transformers that you've already created. This lets you reuse the same modeling process over and over again by wrapping it up in one simple object. Neat, right?
"""

# Instructions

"""

    Import Pipeline from pyspark.ml.
    Call the Pipeline() constructor with the keyword argument stages to create a Pipeline called flights_pipe.
        stages should be a list holding all the stages you want your data to go through in the pipeline. Here this is just: [dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler]

"""

# solution

# Import Pipeline
from pyspark.ml import Pipeline

# Make the pipeline
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler])

#----------------------------------#

# Conclusion

"""
Fantastic! You've made a fully reproducible machine learning pipeline!
"""

"\nFantastic! You've made a fully reproducible machine learning pipeline!\n"

In [19]:
# exercise 09

"""
Transform the data

Hooray, now you're finally ready to pass your data through the Pipeline you created!
"""

# Instructions

"""
Create the DataFrame piped_data by calling the Pipeline methods .fit() and .transform() in a chain. Both of these methods take model_data as their only argument.
"""

# solution

# Fit and transform the data
piped_data = flights_pipe.fit(model_data).transform(model_data)

#----------------------------------#

# Conclusion

"""
Great work! Your pipeline chewed right through that data!
"""

'\nGreat work! Your pipeline chewed right through that data!\n'

In [20]:
# exercise 10

"""
Split the data

Now that you've done all your manipulations, the last step before modeling is to split the data!
"""

# Instructions

"""
Use the DataFrame method .randomSplit() to split piped_data into two pieces, training with 60% of the data, and test with 40% of the data by passing the list [.6, .4] to the .randomSplit() method.
"""

# solution

# Split the data into training and test sets
training, test = piped_data.randomSplit([.6, .4])

#----------------------------------#

# Conclusion

"""
Awesome! Now you're ready to start fitting a model!
"""

"\nAwesome! Now you're ready to start fitting a model!\n"

In [21]:
spark.stop()